In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from statsmodels.formula.api import logit
import statsmodels.formula.api as sm
import warnings
warnings.filterwarnings('ignore')

In [101]:
Pro_Qb_Path = 'Data/QB_Data_Trail.csv'

data = pd.read_csv(Pro_Qb_Path)

players = pd.read_csv('Data/Names.csv')

In [102]:
pd.set_option('display.max_columns', None)

In [103]:
data = data.fillna(0)
data = data.drop(columns=["Unnamed: 0", "Pos_"])

In [104]:
def staus(data):
    different_staus_options = []
    temp = data.drop(columns=["Name_", "Unnamed: 2_level_0_Date", 
                              "Unnamed: 6_level_0_Tm", "Unnamed: 7_level_0_Unnamed: 7_level_1",
                              "Unnamed: 9_level_0_Result", "Unnamed: 8_level_0_Opp" ])
    for x in temp.columns:
        statuses = data[x].unique()
        for w in statuses:
            try:
                float(w)
            except:
                try:
                    float(w[:-1])
                except:
                    different_staus_options.append(w)
                    #print(w)
    return different_staus_options

In [105]:
for index, name in enumerate(data["Name_"]):
    if name == 0:
        data["Name_"][index] = data["Name__"][index]
        data["Name__"][index] = 0
data = data.drop(columns = ["Name__"])

In [106]:
stats = staus(data)

In [107]:
set_res = set(stats) 
list_res = (list(set_res))
values_to_replace = []
 
for item in list_res: 
    print(item)
    values_to_replace.append(item)

Non-Football Injury
Inactive
Injured Reserve
Suspended
COVID-19 List
Did Not Play
Exempt List
*


In [108]:
for x in values_to_replace:
    data[x] = 0

for w in data.columns:
    for index, z in enumerate(data[w]):
        if values_to_replace.count(z) > 0:
            #print(z, index)
            data[z][index] = 1

In [109]:
for x in values_to_replace:
    data = data.replace(x, 0)

In [110]:
for x in data.columns:
    length = len("Unnamed: ")
    if x[:length] == "Unnamed: ":
        real_col = x.split("_")
        data = data.rename(columns={ x : real_col[-1]})
        #print(real_col[-1])
        

In [111]:
#Delete Duplicate columns
data = data.loc[:,~data.columns.duplicated()].copy()

In [112]:
def result_maker(data):
    data["Win?"] = 0
    data["Lose?"] = 0
    data["Tie?"] = 0
    data["Score-not-counted"] = 0
    data["Tm-Score"] = 0
    data["Opp-Score"] = 0
    for index, x in enumerate(data["Result"]):
        if x[0] == "W" or x[0] == "L" or x[0] == "T":
            if x[0] == "W":
                data["Win?"][index] = 1
            elif x[0] == "L":
                data["Lose?"][index] = 1
            else:
                data["Tie?"][index] = 1
            ind_num = x[1:].split("-")
            if len(ind_num) == 2:
                data["Tm-Score"][index] = ind_num[0]
                data["Opp-Score"][index] = ind_num[1]
        else:
            data["Score-not-counted"][index] = 1

In [113]:
result_maker(data)
data = data.drop(columns=["Result"])

In [114]:
def seperate_date(data):
    data["Day"] = 0
    data["Month"] = 0
    data["No-Date"] = 0
    
    for index, x in enumerate(data["Date"]):
        Year_Month_Day = x.split("-")
        if len(Year_Month_Day) > 1:
            Month = Year_Month_Day[1]
            Day = Year_Month_Day[2]
        
            data["Day"][index] = Day
            data["Month"][index] = Month
        else:
            data["No-Date"][index] = 1
    

In [115]:
seperate_date(data)
data = data.drop(columns=["Date"])

In [116]:
def convert_percentage(data):
    for col in data.columns:
        for index, x in enumerate(data[col]):
            try:
                per = x[-1]
                if per == "%": 
                    data[col] = x[:-1]
            except:
                pass

In [117]:
convert_percentage(data)

In [118]:
def convert_to_number(data):
    temp = data.drop(columns = ["Name_", "Tm", "1", "Opp"])
    didnt_work = {}
    
    for col in temp.columns:
        #print(col)
        for x in data[col]:
            try:
                data[col] = x.astype(float)
            except:
                pass
            
        

In [119]:
convert_to_number(data)

In [120]:
def eliminate_Useless(data, num = 1, drop = False):
    limited = []
    for x in data.columns:
        amount = len(data[x].unique())
        if amount <= num:
            limited.append(x)
            print(x, amount, data[x].unique())
    if drop:
        data = data.drop(columns = limited)

In [121]:
eliminate_Useless(data, num = 5, drop = False)

Def Interceptions_Int 5 [0 '0' '1' '2' '27']
Def Interceptions_PD 5 [0 '0' '1' '2' '51']
Def Interceptions_TD 2 [0 '0']
Def. Snaps_Num 3 ['0' 0 '0.0']
Def. Snaps_Pct 1 ['0']
Fumbles_FF 4 ['0' 0 '1' '7']
Fumbles_TD 2 ['0' 0]
Kick Returns_TD 5 [0 '0' '0.0' '1' '1.0']
Off. Snaps_Pct 1 ['100']
Punting_Blck 3 [0 '0' '1']
Punting_In20 4 [0 '0' '1' '3']
Punting_In20% 1 ['0.0']
Punting_RetYds 3 [0 '0' '18']
Punting_TB 4 [0 '0' '1' '2']
Punting_TB% 1 ['100.0']
Receiving_Ctch% 1 ['0.0']
ST Snaps_Pct 1 ['0']
Scoring_2PM 5 [0 '1' '3' '2' '1.0']
Scoring_FG% 4 [  0.  100.   33.3  28.6]
Scoring_FGA 4 [0. 1. 3. 7.]
Scoring_FGM 3 [0. 1. 2.]
Scoring_XP% 4 [  0.   75.  100.   90.9]
Scoring_XPA 5 [ 0.  4.  5.  2. 11.]
Scoring_XPM 5 [ 0.  3.  5.  2. 10.]
Tackles_QBHits 2 [0 '0']
GS 1 [0]
Sk 5 [0 '0.0' '0.5' '1.0' '8.5']
Status 1 [0]
1 2 [0 '@']
Non-Football Injury 2 [0 1]
Inactive 2 [0 1]
Injured Reserve 2 [0 1]
Suspended 2 [0 1]
COVID-19 List 2 [0 1]
Did Not Play 2 [0 1]
Exempt List 2 [0 1]
* 2 [1 0]
Win?

In [122]:
data = data.drop(columns=[
    "Def Interceptions_TD",
    "Def. Snaps_Num",
    "Def. Snaps_Pct",
    "Off. Snaps_Pct",
    "Fumbles_TD",
    "Punting_Blck",
    "Punting_In20%",
    "Punting_TB%",
    "Receiving_Ctch%",
    "ST Snaps_Pct",
    "Tackles_QBHits",
    "GS"
    
])

In [123]:
temp = data.drop(columns = {"Name_", "Tm", "1", "Opp"})
for x in temp.columns:
    if temp[x].dtype == object:
        try:
            data[x] = pd.to_numeric(data[x])
        except:
            print(x)

In [124]:
dk_points = pd.read_csv('Data/WeeklyPoints2014-2021.csv', delimiter = ";")
dk_points

,Week,Year,GID,Name,Pos,Team,h/a,Oppt,DK_points,DK_salary
0,17,2014,1426,"Smith, Geno",QB,nyj,a,mia,30.12,5000
1,17,2014,1408,"Griffin III, Robert",QB,was,h,dal,25.34,5700
2,17,2014,1252,"Rodgers, Aaron",QB,gnb,h,det,24.44,8600
3,17,2014,1302,"Flacco, Joe",QB,bal,h,cle,23.58,6200
4,17,2014,1225,"Manning, Eli",QB,nyg,h,phi,23.16,7100
...,...,...,...,...,...,...,...,...,...,...
55003,1,2021,7002,Atlanta,Def,atl,h,phi,0,2000
55004,1,2021,7006,Chicago,Def,chi,a,lar,0,2800
55005,1,2021,7030,Tennessee,Def,ten,h,ari,0,2300
55006,1,2021,7014,Jacksonville,Def,jac,a,hou,-3,2700


In [125]:
title = ['Week', 'Year', 'GID', "DK_points", "DK_salary"]
index_to_remove = []
for name in title:
    for x in range(len(dk_points[name])):
        try:
            dk_points[name][x] = float(dk_points[name][x])
        except ValueError:
            dk_points[name][x] = 0  
            index_to_remove.append(x)
        

In [126]:
# Replace 'DK points' with NaN (or another value of your choice)
dk_points['Week'] = dk_points['Week'].replace('Week', float('nan'))
dk_points['Week'] = dk_points['Week'].astype(float)


dk_points['Year'] = dk_points['Year'].replace('Year', float('nan'))
dk_points['Year'] = dk_points['Year'].astype(float)

dk_points['DK_points'] = dk_points['DK_points'].replace('DK_points', float('nan'))
dk_points['DK_points'] = dk_points['DK_points'].astype(float)

dk_points['DK_salary'] = dk_points['DK_salary'].replace('DK_salary', float('nan'))
dk_points['DK_salary'] = dk_points['DK_salary'].astype(float)

In [127]:
def get_points(data):
    data["Dk_Points"] = 0
    data["Salary"] = 0
    
    for index, row in dk_points.iterrows():
        Name = row["Name"]
        Week = row["Week"]
        Year = row["Year"]
        
        Index = data[(data["Name_"] == Name) & (data["Year"] == float(Year)) & (data["Week"] == int(Week))].index
        
        data["Dk_Points"][Index] = row["DK_points"]
        data["Salary"][Index] = row["DK_salary"]
    return data

In [128]:
get_points(data)

,Def Interceptions_Int,Def Interceptions_PD,Def Interceptions_Yds,Fumbles_FF,Fumbles_FL,Fumbles_FR,Fumbles_Fmb,Fumbles_Yds,Kick Returns_Rt,Kick Returns_TD,Kick Returns_Y/Rt,Kick Returns_Yds,Name_,Off. Snaps_Num,Passing_AY/A,Passing_Att,Passing_Cmp,Passing_Cmp%,Passing_Int,Passing_Rate,Passing_Sk,Passing_TD,Passing_Y/A,Passing_Yds,Passing_Yds.1,Punt Returns_Ret,Punt Returns_TD,Punt Returns_Y/R,Punt Returns_Yds,Punting_In20,Punting_NY/P,Punting_Net,Punting_Pnt,Punting_RetYds,Punting_TB,Punting_Y/P,Punting_Yds,Receiving_Rec,Receiving_TD,Receiving_Tgt,Receiving_Y/R,Receiving_Y/Tgt,Receiving_Yds,Rushing_Att,Rushing_TD,Rushing_Y/A,Rushing_Yds,ST Snaps_Num,Scoring_2PM,Scoring_FG%,Scoring_FGA,Scoring_FGM,Scoring_Pts,Scoring_TD,Scoring_XP%,Scoring_XPA,Scoring_XPM,Tackles_Ast,Tackles_Comb,Tackles_Solo,Tackles_TFL,Rk,Sk,Year,Status,G#,Week,Age,Tm,1,Opp,Non-Football Injury,Inactive,Injured Reserve,Suspended,COVID-19 List,Did Not Play,Exempt List,*,Win?,Lose?,Tie?,Score-not-counted,Tm-Score,Opp-Score,Day,Month,No-Date,Dk_Points,Salary
0,0,0,0,0,1,0,1,0,0.0,0.0,0.0,0.0,"Smith, Geno",75.0,6.08,38,24,63.16,1,80.6,5,1,6.74,256,42,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,6,0,7.83,47,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1.0,0.0,2013.0,0,1.0,1.0,22.333,NYJ,0,TAM,0,0,0,0,0,0,0,1,1,0,0,0,18,17,8,9,0,0.0,0.0
1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,"Smith, Geno",71.0,2.26,35,15,42.86,3,27.6,4,0,6.11,214,25,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,5.67,17,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,2.0,0.0,2013.0,0,2.0,2.0,22.337,NYJ,@,NWE,0,0,0,0,0,0,0,1,0,1,0,0,10,13,12,9,0,0.0,0.0
2,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,"Smith, Geno",78.0,9.69,29,16,55.17,2,89.9,0,2,11.41,331,0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,3.50,14,0.0,0.0,0.0,0.0,0.0,6,1,0.0,0.0,0.0,0,0,0,0,3.0,0.0,2013.0,0,3.0,3.0,22.347,NYJ,0,BUF,0,0,0,0,0,0,0,1,1,0,0,0,27,20,22,9,0,0.0,0.0
3,0,0,0,0,2,0,2,0,0.0,0.0,0.0,0.0,"Smith, Geno",65.0,6.44,34,23,67.65,2,79.2,5,1,8.50,289,50,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,2.33,7,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,4.0,0.0,2013.0,0,4.0,4.0,22.354,NYJ,@,TEN,0,0,0,0,0,0,0,1,0,1,0,0,13,38,29,9,0,0.0,0.0
4,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,"Smith, Geno",46.0,12.95,20,16,80.00,0,147.7,4,3,9.95,199,29,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,7.00,21,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,5.0,0.0,2013.0,0,5.0,5.0,22.362,NYJ,@,ATL,0,0,0,0,0,0,0,1,1,0,0,0,30,28,7,10,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33248,0,0,0,0,0,1,1,0,0.0,0.0,0.0,0.0,"Burrow, Joe",66.0,6.03,39,27,69.23,1,104.6,2,4,5.13,200,16,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,2.67,8,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,47.0,0.0,2022.0,0,14.0,15.0,26.008,CIN,@,TAM,0,0,0,0,0,0,0,1,1,0,0,0,34,23,18,12,0,0.0,0.0
33249,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,"Burrow, Joe",83.0,6.63,52,40,76.92,2,99.4,2,3,7.21,375,6,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,4,0,1.25,5,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,48.0,0.0,2022.0,0,15.0,16.0,26.014,CIN,@,NWE,0,0,0,0,0,0,0,1,1,0,0,0,22,18,24,12,0,0.0,0.0
33250,0,0,0,0,1,0,1,0,0.0,0.0,0.0,0.0,"Burrow, Joe",66.0,5.60,42,25,59.52,0,81.0,2,1,5.12,215,13,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,3.33,10,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,49.0,0.0,2022.0,0,16.0,18.0,26.029,CIN,0,BAL,0,0,0,0,0,0,0,1,1,0,0,0,27,16,8,1,0,0.0,0.0
33251,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,"Burrow, Joe",52.0,2.65,31,14,45.16,0,52.2,2,0,2.65,82,15,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,-1.00,-1,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0

In [129]:
#names = pd.DataFrame(data={"Name_":data["Name_"].unique()})
#print(data["Name_"].unique(), "\n", data["Name_"])
#names.to_csv('filtered_data_RB.csv', index=False, sep=";")

In [130]:
def filter_for_Prep(data):
    
    filtered_data = data[data["Year"] >= 2014]
    filtered_data = filtered_data.drop(columns = ["Rk", "G#"])
    filtered_data = filtered_data.loc[:,~data.T.duplicated(keep='first')]
    do_not_drop = ['Injured Reserve','Non-Football Injury','Suspended','Exempt List','Did Not Play','Physically Unable to Perform','Inactive','COVID-19 List','Tie?','Score-not-counted']
    eliminated = []
    eliminated_value = []
    for col in filtered_data.columns:
        num = (filtered_data[col] == 0).sum()/len(filtered_data)
        
        if num > 0.89 and do_not_drop.count(col) == 0:
            eliminated.append(col)
            eliminated_value.append(num)
            filtered_data = filtered_data.drop(columns=[col])
        
        
        
            
    return filtered_data, eliminated

In [131]:
filtered_data, eliminated = filter_for_Prep(data)

In [132]:
filtered_data

,Fumbles_Fmb,Name_,Off. Snaps_Num,Passing_AY/A,Passing_Att,Passing_Cmp,Passing_Cmp%,Passing_Int,Passing_Rate,Passing_Sk,Passing_TD,Passing_Y/A,Passing_Yds,Passing_Yds.1,Rushing_Att,Rushing_Y/A,Rushing_Yds,Year,Week,Age,Tm,1,Opp,Non-Football Injury,Inactive,Injured Reserve,Suspended,COVID-19 List,Did Not Play,Exempt List,*,Win?,Lose?,Tie?,Score-not-counted,Tm-Score,Opp-Score,Day,Month,Dk_Points,Salary
16,2,"Smith, Geno",70.0,7.00,28,23,82.14,1,96.6,2,1,7.89,221,31,10,3.80,38,2014.0,1.0,23.332,NYJ,0,OAK,0,0,0,0,0,0,0,1,1,0,0,0,19,14,7,9,14.64,6500.0
17,0,"Smith, Geno",71.0,4.72,32,16,50.00,1,64.1,1,1,5.50,176,7,7,3.71,26,2014.0,2.0,23.339,NYJ,@,GNB,0,0,0,0,0,0,0,1,0,1,0,0,24,31,14,9,18.64,6200.0
18,0,"Smith, Geno",74.0,5.72,43,26,60.47,2,71.5,3,1,7.35,316,16,3,6.67,20,2014.0,3.0,23.347,NYJ,0,CHI,0,0,0,0,0,0,0,1,0,1,0,0,19,27,22,9,19.64,6100.0
19,2,"Smith, Geno",64.0,5.58,33,17,51.52,1,68.9,2,1,6.33,209,5,4,1.00,4,2014.0,4.0,23.353,NYJ,0,DET,0,0,0,0,0,0,0,1,0,1,0,0,17,24,28,9,10.76,6500.0
20,0,"Smith, Geno",23.0,-1.50,12,4,33.33,1,7.6,0,0,2.25,27,0,1,3.00,3,2014.0,5.0,23.360,NYJ,@,SDG,0,0,0,0,0,0,0,1,0,1,0,0,0,31,5,10,0.38,6700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33247,0,"Burrow, Joe",67.0,7.09,33,18,54.55,1,85.3,1,2,7.24,239,3,6,1.67,10,2022.0,14.0,26.001,CIN,0,CLE,0,0,0,0,0,0,0,1,1,0,0,0,23,10,11,12,0.00,0.0
33248,1,"Burrow, Joe",66.0,6.03,39,27,69.23,1,104.6,2,4,5.13,200,16,3,2.67,8,2022.0,15.0,26.008,CIN,@,TAM,0,0,0,0,0,0,0,1,1,0,0,0,34,23,18,12,0.00,0.0
33249,0,"Burrow, Joe",83.0,6.63,52,40,76.92,2,99.4,2,3,7.21,375,6,4,1.25,5,2022.0,16.0,26.014,CIN,@,NWE,0,0,0,0,0,0,0,1,1,0,0,0,22,18,24,12,0.00,0.0
33250,1,"Burrow, Joe",66.0,5.60,42,25,59.52,0,81.0,2,1,5.12,215,13,3,3.33,10,2022.0,18.0,26.029,CIN,0,BAL,0,0,0,0,0,0,0,1,1,0,0,0,27,16,8,1,0.00,0.0


In [133]:
heights_and_weights = pd.read_csv('Data/data_RBs.csv', sep = ";")

heights_and_weights["Feet"] = 0
heights_and_weights["Inches"] = 0
heights_and_weights["Weight(lbs)"] = 0

index = 0
for x in heights_and_weights["Info"]:
    feet = x[0]
    inch = x[2]
    weight = x[5:8]
    
    heights_and_weights["Feet"][index] = feet
    heights_and_weights["Inches"][index] = inch
    heights_and_weights["Weight(lbs)"][index] = weight
    index += 1

In [134]:
index = 0
filtered_data = filtered_data.reset_index(drop=True)
filtered_data["Feet"] = 0
filtered_data["Inches"] = 0
filtered_data["Weight(lbs)"] = 0

dumb_players_with_no_weight = []
dumb_players_with_no_weight_index = []
for index, x in enumerate(filtered_data["Name_"]):
    try:
        indices = heights_and_weights[heights_and_weights['Name'] == x].index[0]

        filtered_data["Feet"][index] = heights_and_weights["Feet"][indices]
        filtered_data["Inches"][index] = heights_and_weights["Inches"][indices]
        filtered_data["Weight(lbs)"][index] = heights_and_weights["Weight(lbs)"][indices]
    except:
        dumb_players_with_no_weight.append(x)
        dumb_players_with_no_weight_index.append(index)
        
    index += 1

    
filtered_data.drop(labels = dumb_players_with_no_weight_index, axis = 0)
filtered_data = filtered_data.reset_index(drop=True)

In [136]:
Extra_Weights = pd.read_csv('Data/Extra_Weight and hieght - Sheet1.csv')
Extra_Weights["Feet"] = 0
Extra_Weights["Inches"] = 0

for index, row in Extra_Weights.iterrows():
    Extra_Weights["Weight"][index] = Extra_Weights["Weight"][index][:3]
    Extra_Weights["Feet"][index] = Extra_Weights["Height"][index][0]
    Extra_Weights["Inches"][index] = Extra_Weights["Height"][index].split("'")[1][:-1]

In [137]:
name_options = list(Extra_Weights.Player)

no_name = []
for index, foot in enumerate(filtered_data["Feet"]):
    if foot == 0:
        name = filtered_data["Name_"][index]
        if name in name_options:
            
            indices = Extra_Weights.index[Extra_Weights['Player'] == name].tolist()
            
            
            filtered_data["Weight(lbs)"][index] = Extra_Weights["Weight"][indices]
            filtered_data["Feet"][index] = Extra_Weights["Feet"][indices]
            filtered_data["Inches"][index] = Extra_Weights["Inches"][indices]
        
        else:
            no_name.append(name)

no_name

['Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco, Joe',
 'Flacco

In [139]:
temp = pd.DataFrame(data = {"Name_" : no_name})
temp.Name_.unique()

array(['Flacco, Joe', 'Manning, Eli', 'Kaepernick, Colin',
       'Roethlisberger, Ben', 'Sanchez, Mark', 'Lindley, Ryan',
       'Newton, Cam', 'Romo, Tony', 'Luck, Andrew', 'Brees, Drew',
       'Cutler, Jay', 'Bortles, Blake', 'Orton, Kyle', 'Manning, Peyton',
       'Hasselbeck, Matt', 'Ryan, Matt', 'Rivers, Philip',
       'Daniel, Chase', 'Hill, Shaun', 'Shaw, Connor', 'Osweiler, Brock',
       'Brady, Tom', 'Flynn, Matt', 'Palmer, Jordan', 'Schaub, Matt',
       'McCown, Luke', 'Webb, Joe', 'Vick, Michael', 'Weeden, Brandon',
       'Clausen, Jimmy', 'Manziel, Johnny', 'Jackson, Tarvaris',
       'Anderson, Derek', 'Stanton, Drew', 'McCoy, Colt', 'Locker, Jake',
       'Fitzpatrick, Ryan', 'Savage, Tom', 'Mettenberger, Zach',
       'Campbell, Jason', 'Barkley, Matt', 'Nassib, Ryan',
       'Davis, Austin', 'Mallett, Ryan', 'Palmer, Carson',
       'Glennon, Mike', 'Foles, Nick', 'Simms, Matt', 'Clemens, Kellen',
       'Gradkowski, Bruce', 'Ponder, Christian', 'Cassel, Matt',
 

In [38]:
Fan_Points_2013_2022 = pd.read_csv('Data/Fantasy_Points_2013-2022 - Sheet1.csv')
Fan_Points_2013_2022.TEAM = Fan_Points_2013_2022.TEAM.replace("LV", "LVR").replace("JAC", "JAX").replace("SF", "SFO").replace("TB", "TAM").replace("GB", "GNB").replace("KC", "KAN").replace("NO", "NOR").replace("NE", "NWE")

In [39]:
Fan_Points_2013_2022 = Fan_Points_2013_2022[Fan_Points_2013_2022["POS"] == "QB"].reset_index(drop=True)
Fan_Points_2013_2022["Convert_Name"] = 0


for index, name in enumerate(Fan_Points_2013_2022["PLAYER"]):
    name_seg = name.split(" ")
    if len(name_seg) == 2:
        Fan_Points_2013_2022["Convert_Name"][index] = name_seg[1] + ", " + name_seg[0]
    else:
        Fan_Points_2013_2022["Convert_Name"][index] = name_seg[1] + " " + name_seg[2] + ", " + name_seg[0]

In [40]:
def place_2022_scoring(filtered_data, Fan_Points_2013_2022):
    name_options = list(filtered_data["Name_"].unique())
    idoit = []
    accounted = 0
    amount = 0
    for index, row in Fan_Points_2013_2022.iterrows():
        name = row["Convert_Name"]
        week = 1
        if row["Year"] >= 2021:
            Games = row[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']]
        else:
            Games = row[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17']]
        if name in name_options:
            #print(name)
            for index, col in enumerate(Games.keys(), start=1):
                if col in Games:
                    try:
                        points = float(Games[col])
                    except:
                        points = 0
                        accounted += 1
                        
                    filtered_data.loc[(filtered_data["Year"] == int(row["Year"])) & (filtered_data["Name_"] == name) & (filtered_data["Week"] == week), 'Dk_Points'] = points
                amount += 1
                week += 1
        else:
            idoit.append(name)
    print(len(idoit)/len(list(filtered_data["Name_"].unique())) * 100, "% of the points sheet are not in", "\n", accounted/amount * 100, "% was accounted for")
    filtered_data["Dk_Points"] = filtered_data["Dk_Points"].astype(float)
    return idoit

In [41]:
idoit = place_2022_scoring(filtered_data, Fan_Points_2013_2022)


8.19672131147541 % of the points sheet are not in 
 49.43036731486938 % was accounted for


In [42]:
temp = pd.DataFrame(data = {"Name_" : idoit})
list(temp["Name_"].unique())

['Corral, Matt',
 'Cheek, Davis',
 'Stevens, Tommy',
 'Lovett, John',
 'Webb III, Joe',
 'Griffin, Ryan',
 'Franklin, James',
 'Sloter, Kyle']

In [189]:
# more_forgotten_RB = {
#     "Burrow, Joe" : "https://www.pro-football-reference.com/players/B/BurrJo01/gamelog/",
#     "Jones, Daniel" : "https://www.pro-football-reference.com/players/J/JoneDa05/gamelog/",
#     "Prescott, Dak" : "https://www.pro-football-reference.com/players/P/PresDa01/gamelog/",
#     "Jones, Mac" : "https://www.pro-football-reference.com/players/J/JoneMa05/gamelog/",
#     "Walker, PJ" : "https://www.pro-football-reference.com/players/W/WalkPh00/gamelog/",
#     "Willis, Malik" : "https://www.pro-football-reference.com/players/W/WillMa12/gamelog/",
#     "Perkins, Bryce" : "https://www.pro-football-reference.com/players/P/PerkBr02/gamelog/",
#     "Love, Jordan" : "https://www.pro-football-reference.com/players/L/LoveJo03/gamelog/",
#     "Brown, Anthony" : "https://www.pro-football-reference.com/players/B/BrowAn06/gamelog/",
#     "Johnson, Josh" : "https://www.pro-football-reference.com/players/J/JohnJo05/gamelog/",
#     "Henne, Chad" : "https://www.pro-football-reference.com/players/H/HennCh01/gamelog/",
#     "Gilbert, Garrett" : "https://www.pro-football-reference.com/players/G/GilbGa01/gamelog/",
#     "Rosen, Josh" : "https://www.pro-football-reference.com/players/R/RoseJo01/gamelog/",
#     "Smith, Alex" : "https://www.pro-football-reference.com/players/S/SmitAl03/gamelog/",
#     "Griffin III, Robert" : "https://www.pro-football-reference.com/players/G/GrifRo01/gamelog/",
#     "McCarron, AJ" : "https://www.pro-football-reference.com/players/M/McCaA.00/gamelog/",
#     "Moore, Matt" : "https://www.pro-football-reference.com/players/M/MoorMa01/gamelog/",
#     "McCown, Josh" : "https://www.pro-football-reference.com/players/M/McCoJo01/gamelog/"
    
# }

In [43]:
# last = []
# first = []

# for x in idoit:
#     sep = x.split(", ")
#     last.append(sep[0])
#     first.append(sep[1])

In [44]:
# forgotten = pd.DataFrame(data={"Name_":idoit, "First Name": first, "Last Name": last})
# #print(data["Name_"].unique(), "\n", data["Name_"])
# forgotten.to_csv('Forgotten_QB.csv', index=False, sep=";")

In [45]:
Offesive_Team_Stats = pd.read_csv('Data/NFL Team Season Stats - Offense Reg Season.csv', thousands=',')
Defensive_Team_Stats = pd.read_csv('Data/NFL Team Season Stats - Defense Reg Season.csv', thousands=',')
filtered_data["Tm"] = filtered_data["Tm"].replace("STL", "NOR").replace("OAK", "LVR").replace("SDG", "LAC")

In [46]:
def offensive_scoring_creation(filtered_data):
    filtered_data["Last_Off_Total_YDS"] = 0
    filtered_data["Last_Off_Total_YDS/G"] = 0
    filtered_data["Last_Off_Passing_YDS"] = 0
    filtered_data["Last_Off_Passing_YDS/G"] = 0
    filtered_data["Last_Off_Rushing_YDS"] = 0
    filtered_data["Last_Off_Rushing_YDS/G"] = 0
    filtered_data["Last_Off_Points_PTS"] = 0
    filtered_data["Last_Off_PointsPTS/G"] = 0
    for x in filtered_data.index:
        Team = filtered_data.loc[x]["Tm"]
        Year = filtered_data.loc[x]["Year"]
        row = Offesive_Team_Stats[(Offesive_Team_Stats["TEAM"] == Team) & (Offesive_Team_Stats["Year"] == Year)]

        # Ensure the conditions return only one row
        if len(row) == 1:
            filtered_data.loc[x, "Last_Off_Total_YDS"] = row["Total_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Off_Total_YDS/G"] = row["Total_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Off_Passing_YDS"] = row["Passing_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Off_Passing_YDS/G"] = row["Passing_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Off_Rushing_YDS"] = row["Rushing_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Off_Rushing_YDS/G"] = row["Rushing_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Off_Points_PTS"] = row["Points_PTS"].iloc[0]
            filtered_data.loc[x, "Last_Off_PointsPTS/G"] = row["PointsPTS/G"].iloc[0]

In [47]:
def def_scoring_creation(filtered_data):
    filtered_data["Last_Def_Total_YDS"] = 0
    filtered_data["Last_Def_Total_YDS/G"] = 0
    filtered_data["Last_Def_Passing_YDS"] = 0
    filtered_data["Last_Def_Passing_YDS/G"] = 0
    filtered_data["Last_Def_Rushing_YDS"] = 0
    filtered_data["Last_Def_Rushing_YDS/G"] = 0
    filtered_data["Last_Def_Points_PTS"] = 0
    filtered_data["Last_Def_PointsPTS/G"] = 0
    for x in filtered_data.index:
        Team = filtered_data.loc[x]["Tm"]
        Year = filtered_data.loc[x]["Year"]
        row = Defensive_Team_Stats[(Defensive_Team_Stats["TEAM"] == Team) & (Defensive_Team_Stats["Year"] == Year)]

        # Ensure the conditions return only one row
        if len(row) == 1:
            filtered_data.loc[x, "Last_Def_Total_YDS"] = row["Total_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Def_Total_YDS/G"] = row["Total_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Def_Passing_YDS"] = row["Passing_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Def_Passing_YDS/G"] = row["Passing_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Def_Rushing_YDS"] = row["Rushing_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Def_Rushing_YDS/G"] = row["Rushing_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Def_Points_PTS"] = row["Points_PTS"].iloc[0]
            filtered_data.loc[x, "Last_Def_PointsPTS/G"] = row["PointsPTS/G"].iloc[0]

In [48]:
#delete duplicate rows
filtered_data = filtered_data.drop_duplicates(keep='last')

In [49]:
def_scoring_creation(filtered_data)
offensive_scoring_creation(filtered_data)

In [140]:
def wr_week_one(data, team, opp, players, year):
    
    summary_team_stats = pd.DataFrame(data={"Last_Off_Total_YDS" : [0], 
                                            "Last_Off_Total_YDS/G" : [0], 
                                            "Last_Off_Passing_YDS" : [0],
                                            "Last_Off_Passing_YDS/G" : [0], 
                                            "Last_Off_Rushing_YDS" : [0], 
                                            "Last_Off_Rushing_YDS/G" : [0], 
                                            "Last_Off_Points_PTS" : [0], 
                                            "Last_Off_PointsPTS/G" : [0], 
                                            "Last_Def_Total_YDS" : [0],
                                            "Last_Def_Total_YDS/G" : [0], 
                                            "Last_Def_Passing_YDS" : [0], 
                                            "Last_Def_Passing_YDS/G" : [0], 
                                            "Last_Def_Rushing_YDS" : [0], 
                                            "Last_Def_Rushing_YDS/G" : [0], 
                                            "Last_Def_Points_PTS" : [0],
                                            "Last_Def_PointsPTS/G" : [0]})
    
    temp = pd.DataFrame(data = {"Name_":players})
    drop = ["Name_", "Tm", "1", "Opp", "Week", "Year", "Day", 
            'Tie?','Score-not-counted', "Weight(lbs)", "Inches", "Feet",
           "Last_Off_Total_YDS", "Last_Off_Total_YDS/G", "Last_Off_Passing_YDS",
           "Last_Off_Passing_YDS/G", "Last_Off_Rushing_YDS", "Last_Off_Rushing_YDS/G", 
           "Last_Off_Points_PTS", "Last_Off_PointsPTS/G", "Last_Def_Total_YDS",
           "Last_Def_Total_YDS/G", "Last_Def_Passing_YDS", "Last_Def_Passing_YDS/G", 
           "Last_Def_Rushing_YDS", "Last_Def_Rushing_YDS/G", "Last_Def_Points_PTS",
           "Last_Def_PointsPTS/G"]
    #temp = data.drop(columns = ["Name_", "Tm", "1", "Opp"])
    used_data = data.drop(columns = drop)
    
    for col in used_data.columns:
            if used_data[col].dtype != object:
                #temp[col + "_Count"] = 0
                temp[col + "_Average"] = 0
                temp[col + "_STD"] = 0
                temp[col + "_Min"] = 0
                temp[col + "_25%"] = 0
                temp[col + "_50%"] = 0
                temp[col + "_100%"] = 0
                temp[col + "_Max"] = 0
                
                
                #temp["History_Opp_" + col + "_Count"] = 0
                temp["History_" + col + "_Average"] = 0
                temp["History_" + col + "_STD"] = 0
                temp["History_" + col + "_Min"] = 0
                temp["History_" + col + "_25%"] = 0
                temp["History_" + col + "_50%"] = 0
                temp["History_" + col + "_100%"] = 0
                temp["History_" + col + "_Max"] = 0
    
    
    
    seperate_df = list(summary_team_stats.columns)
    
    temp["Feet"] = 0
    temp["Inches"] = 0
    temp["Weight(lbs)"] = 0
    temp["Actaul_Dk_Points"] = 0    
                
    sub_data = data[data["Year"] > 2013]
    #History of all games per player on team   
    index = 0
    for name in players:
        indiv_player = data[(data["Name_"] == name) & (data["Year"] < year)]
        indiv_player = indiv_player.drop(columns = drop)
       
        
        for col in indiv_player:
            if seperate_df.count(col) == 0:
                stats = indiv_player[col].describe()
                #temp[col + "_Count"][index] = stats[0]
                temp[col + "_Average"][index] = stats[1]
                temp[col + "_STD"][index] = stats[2]
                temp[col + "_Min"][index] = stats[3]
                temp[col + "_25%"][index] = stats[4]
                temp[col + "_50%"][index] = stats[5]
                temp[col + "_100%"][index] = stats[6]
                temp[col + "_Max"][index] = stats[7]   
        index += 1
    
    
    #History of all players on team vs oppenant
    index = 0 
    for name in players:
        last_games = data[(data["Name_"] == name) & (data["Opp"] == opp) & (data["Year"] < year)]
        last_games = last_games.drop(columns = drop)
        
        for col in last_games:
            if seperate_df.count(col) == 0:
                stats = last_games[col].describe()
                #temp["History_Opp_" + col + "_Count"][index] = stats[0]
                temp["History_" + col + "_Average"][index] = stats[1]
                temp["History_" + col + "_STD"][index] = stats[2]
                temp["History_" + col + "_Min"][index] = stats[3]
                temp["History_" + col + "_25%"][index] = stats[4]
                temp["History_" + col + "_50%"][index] = stats[5]
                temp["History_" + col + "_100%"][index] = stats[6]
                temp["History_" + col + "_Max"][index] = stats[7]   
        index += 1
    
    temp["Opp"] = opp
    temp["Team"] = team
    
    
    
    
    #Get the real points they made and bio
    index = 0
    for name in players:
        
        test_game = data[(data["Name_"] == name) & (data["Opp"] == opp) & (data["Year"] == year) & (data["Week"] == 1)]

        # Assuming test_game contains only one row, otherwise this will error out.
        if len(test_game) == 1:
            #print(test_game["Dk_Points"].iloc[0])

            temp.loc[index, "Actaul_Dk_Points"] = test_game["Dk_Points"].iloc[0]
            temp.loc[index, "Feet"] = test_game["Feet"].iloc[0]
            temp.loc[index, "Inches"] = test_game["Inches"].iloc[0]
            temp.loc[index, "Weight(lbs)"] = test_game["Weight(lbs)"].iloc[0]
        else:
            # Handle cases where test_game might have more than one row or no rows.
            print(f"Expected only one row for conditions, but found {len(test_game)} rows.")
        
        index += 1
    
    summary_team_stats["Last_Off_Total_YDS"] = float(test_game["Last_Off_Total_YDS"])
    summary_team_stats["Last_Off_Total_YDS/G"] = float(test_game["Last_Off_Total_YDS"])
    summary_team_stats["Last_Off_Passing_YDS"] = float(test_game["Last_Off_Passing_YDS"])
    summary_team_stats["Last_Off_Passing_YDS/G"] = float(test_game["Last_Off_Passing_YDS/G"])
    summary_team_stats["Last_Off_Rushing_YDS"] = float(test_game["Last_Off_Rushing_YDS"])
    summary_team_stats["Last_Off_Rushing_YDS/G"] = float(test_game["Last_Off_Rushing_YDS/G"])
    summary_team_stats["Last_Off_Points_PTS"] = float(test_game["Last_Off_Points_PTS"])
    summary_team_stats["Last_Off_PointsPTS/G"] = float(test_game["Last_Off_PointsPTS/G"])
    
    summary_team_stats["Last_Def_Total_YDS"] = float(test_game["Last_Def_Total_YDS"])
    summary_team_stats["Last_Def_Total_YDS/G"] = float(test_game["Last_Def_Total_YDS/G"])
    summary_team_stats["Last_Def_Passing_YDS"] = float(test_game["Last_Def_Passing_YDS"])
    summary_team_stats["Last_Def_Passing_YDS/G"] = float(test_game["Last_Def_Passing_YDS/G"])
    summary_team_stats["Last_Def_Rushing_YDS"] = float(test_game["Last_Def_Rushing_YDS"])
    summary_team_stats["Last_Def_Rushing_YDS/G"] = float(test_game["Last_Def_Rushing_YDS/G"])
    summary_team_stats["Last_Def_Points_PTS"] = float(test_game["Last_Def_Points_PTS"])
    summary_team_stats["Last_Def_PointsPTS/G"] = float(test_game["Last_Def_PointsPTS/G"])
        
        
        
        
        
    
    temp = temp.fillna(0)
    
    
    return temp, summary_team_stats

In [141]:
ready_data, team_data = wr_week_one(filtered_data, "CIN", "PIT", ["Burrow, Joe"], year = 2022)

KeyError: "['Last_Off_Total_YDS' 'Last_Off_Total_YDS/G' 'Last_Off_Passing_YDS'\n 'Last_Off_Passing_YDS/G' 'Last_Off_Rushing_YDS' 'Last_Off_Rushing_YDS/G'\n 'Last_Off_Points_PTS' 'Last_Off_PointsPTS/G' 'Last_Def_Total_YDS'\n 'Last_Def_Total_YDS/G' 'Last_Def_Passing_YDS' 'Last_Def_Passing_YDS/G'\n 'Last_Def_Rushing_YDS' 'Last_Def_Rushing_YDS/G' 'Last_Def_Points_PTS'\n 'Last_Def_PointsPTS/G'] not found in axis"

In [73]:
ready_data

,Name_,Fumbles_Fmb_Average,Fumbles_Fmb_STD,Fumbles_Fmb_Min,Fumbles_Fmb_25%,Fumbles_Fmb_50%,Fumbles_Fmb_100%,Fumbles_Fmb_Max,History_Fumbles_Fmb_Average,History_Fumbles_Fmb_STD,History_Fumbles_Fmb_Min,History_Fumbles_Fmb_25%,History_Fumbles_Fmb_50%,History_Fumbles_Fmb_100%,History_Fumbles_Fmb_Max,Off. Snaps_Num_Average,Off. Snaps_Num_STD,Off. Snaps_Num_Min,Off. Snaps_Num_25%,Off. Snaps_Num_50%,Off. Snaps_Num_100%,Off. Snaps_Num_Max,History_Off. Snaps_Num_Average,History_Off. Snaps_Num_STD,History_Off. Snaps_Num_Min,History_Off. Snaps_Num_25%,History_Off. Snaps_Num_50%,History_Off. Snaps_Num_100%,History_Off. Snaps_Num_Max,Passing_AY/A_Average,Passing_AY/A_STD,Passing_AY/A_Min,Passing_AY/A_25%,Passing_AY/A_50%,Passing_AY/A_100%,Passing_AY/A_Max,History_Passing_AY/A_Average,History_Passing_AY/A_STD,History_Passing_AY/A_Min,History_Passing_AY/A_25%,History_Passing_AY/A_50%,History_Passing_AY/A_100%,History_Passing_AY/A_Max,Passing_Att_Average,Passing_Att_STD,Passing_Att_Min,Passing_Att_25%,Passing_Att_50%,Passing_Att_100%,Passing_Att_Max,History_Passing_Att_Average,History_Passing_Att_STD,History_Passing_Att_Min,History_Passing_Att_25%,History_Passing_Att_50%,History_Passing_Att_100%,History_Passing_Att_Max,Passing_Cmp_Average,Passing_Cmp_STD,Passing_Cmp_Min,Passing_Cmp_25%,Passing_Cmp_50%,Passing_Cmp_100%,Passing_Cmp_Max,History_Passing_Cmp_Average,History_Passing_Cmp_STD,History_Passing_Cmp_Min,History_Passing_Cmp_25%,History_Passing_Cmp_50%,History_Passing_Cmp_100%,History_Passing_Cmp_Max,Passing_Cmp%_Average,Passing_Cmp%_STD,Passing_Cmp%_Min,Passing_Cmp%_25%,Passing_Cmp%_50%,Passing_Cmp%_100%,Passing_Cmp%_Max,History_Passing_Cmp%_Average,History_Passing_Cmp%_STD,History_Passing_Cmp%_Min,History_Passing_Cmp%_25%,History_Passing_Cmp%_50%,History_Passing_Cmp%_100%,History_Passing_Cmp%_Max,Passing_Int_Average,Passing_Int_STD,Passing_Int_Min,Passing_Int_25%,Passing_Int_50%,Passing_Int_100%,Passing_Int_Max,History_Passing_Int_Average,History_Passing_Int_STD,History_Passing_Int_Min,History_Passing_Int_25%,History_Passing_Int_50%,History_Passing_Int_100%,History_Passing_Int_Max,Passing_Rate_Average,Passing_Rate_STD,Passing_Rate_Min,Passing_Rate_25%,Passing_Rate_50%,Passing_Rate_100%,Passing_Rate_Max,History_Passing_Rate_Average,History_Passing_Rate_STD,History_Passing_Rate_Min,History_Passing_Rate_25%,History_Passing_Rate_50%,History_Passing_Rate_100%,History_Passing_Rate_Max,Passing_Sk_Average,Passing_Sk_STD,Passing_Sk_Min,Passing_Sk_25%,Passing_Sk_50%,Passing_Sk_100%,Passing_Sk_Max,History_Passing_Sk_Average,History_Passing_Sk_STD,History_Passing_Sk_Min,History_Passing_Sk_25%,History_Passing_Sk_50%,History_Passing_Sk_100%,History_Passing_Sk_Max,Passing_TD_Average,Passing_TD_STD,Passing_TD_Min,Passing_TD_25%,Passing_TD_50%,Passing_TD_100%,Passing_TD_Max,History_Passing_TD_Average,History_Passing_TD_STD,History_Passing_TD_Min,History_Passing_TD_25%,History_Passing_TD_50%,History_Passing_TD_100%,History_Passing_TD_Max,Passing_Y/A_Average,Passing_Y/A_STD,Passing_Y/A_Min,Passing_Y/A_25%,Passing_Y/A_50%,Passing_Y/A_100%,Passing_Y/A_Max,History_Passing_Y/A_Average,History_Passing_Y/A_STD,History_Passing_Y/A_Min,History_Passing_Y/A_25%,History_Passing_Y/A_50%,History_Passing_Y/A_100%,History_Passing_Y/A_Max,Passing_Yds_Average,Passing_Yds_STD,Passing_Yds_Min,Passing_Yds_25%,Passing_Yds_50%,Passing_Yds_100%,Passing_Yds_Max,History_Passing_Yds_Average,History_Passing_Yds_STD,History_Passing_Yds_Min,History_Passing_Yds_25%,History_Passing_Yds_50%,History_Passing_Yds_100%,History_Passing_Yds_Max,Passing_Yds.1_Average,Passing_Yds.1_STD,Passing_Yds.1_Min,Passing_Yds.1_25%,Passing_Yds.1_50%,Passing_Yds.1_100%,Passing_Yds.1_Max,History_Passing_Yds.1_Average,History_Passing_Yds.1_STD,History_Passing_Yds.1_Min,History_Passing_Yds.1_25%,History_Passing_Yds.1_50%,History_Passing_Yds.1_100%,History_Passing_Yds.1_Max,Rushing_Att_Average,Rushing_Att_STD,Rushing_Att_Min,Rushing_Att_25%,Rushing_Att_50%,Rushing_Att_100%,Rushing_Att_Max,History_Rushing_Att_Av

In [74]:
team_data

,Last_Off_Total_YDS,Last_Off_Total_YDS/G,Last_Off_Passing_YDS,Last_Off_Passing_YDS/G,Last_Off_Rushing_YDS,Last_Off_Rushing_YDS/G,Last_Off_Points_PTS,Last_Off_PointsPTS/G,Last_Def_Total_YDS,Last_Def_Total_YDS/G,Last_Def_Passing_YDS,Last_Def_Passing_YDS/G,Last_Def_Rushing_YDS,Last_Def_Rushing_YDS/G,Last_Def_Points_PTS,Last_Def_PointsPTS/G
0,6034.0,6034.0,3736.0,219.8,2298.0,135.2,467.0,27.5,5613.0,330.2,3415.0,200.9,2198.0,129.3,342.0,20.1


In [54]:
filtered_data[(filtered_data["Name_"] == "Burrow, Joe") & (filtered_data["Year"] == 2022)]

,Fumbles_Fmb,Name_,Off. Snaps_Num,Passing_AY/A,Passing_Att,Passing_Cmp,Passing_Cmp%,Passing_Int,Passing_Rate,Passing_Sk,Passing_TD,Passing_Y/A,Passing_Yds,Passing_Yds.1,Rushing_Att,Rushing_Y/A,Rushing_Yds,Year,Week,Age,Tm,1,Opp,Non-Football Injury,Inactive,Injured Reserve,Suspended,COVID-19 List,Did Not Play,Exempt List,*,Win?,Lose?,Tie?,Score-not-counted,Tm-Score,Opp-Score,Day,Month,Dk_Points,Salary,Feet,Inches,Weight(lbs),Last_Def_Total_YDS,Last_Def_Total_YDS/G,Last_Def_Passing_YDS,Last_Def_Passing_YDS/G,Last_Def_Rushing_YDS,Last_Def_Rushing_YDS/G,Last_Def_Points_PTS,Last_Def_PointsPTS/G,Last_Off_Total_YDS,Last_Off_Total_YDS/G,Last_Off_Passing_YDS,Last_Off_Passing_YDS/G,Last_Off_Rushing_YDS,Last_Off_Rushing_YDS/G,Last_Off_Points_PTS,Last_Off_PointsPTS/G
18151,2,"Burrow, Joe",100.0,3.74,53,33,62.26,4,61.7,7,2,6.38,338,39,6,7.83,47,2022.0,1.0,25.275,CIN,0,PIT,0,0,0,0,0,0,0,1,0,1,0,0,20,23,11,9,22.2,0.0,0,0,0,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
18152,1,"Burrow, Joe",70.0,6.08,36,24,66.67,0,89.9,6,1,5.53,199,34,4,6.50,26,2022.0,2.0,25.282,CIN,@,DAL,0,0,0,0,0,0,0,1,0,1,0,0,17,20,18,9,16.6,0.0,0,0,0,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
18153,0,"Burrow, Joe",68.0,9.31,36,23,63.89,0,114.9,2,3,7.64,275,14,5,0.00,0,2022.0,3.0,25.289,CIN,@,NYJ,0,0,0,0,0,0,0,1,1,0,0,0,27,12,25,9,23.0,0.0,0,0,0,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
18154,0,"Burrow, Joe",64.0,10.55,31,20,64.52,0,115.9,1,2,9.26,287,6,5,1.20,6,2022.0,4.0,25.293,CIN,0,MIA,0,0,0,0,0,0,0,1,1,0,0,0,27,15,29,9,20.1,0.0,0,0,0,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
18155,0,"Burrow, Joe",63.0,5.49,35,24,68.57,1,82.7,2,1,6.20,217,15,3,2.00,6,2022.0,5.0,25.303,CIN,@,BAL,0,0,0,0,0,0,0,1,0,1,0,0,17,19,9,10,18.3,0.0,0,0,0,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
18156,0,"Burrow, Joe",57.0,9.73,37,28,75.68,0,126.0,3,3,8.11,300,27,4,6.25,25,2022.0,6.0,25.310,CIN,@,NOR,0,0,0,0,0,0,0,1,1,0,0,0,30,26,16,10,32.5,0.0,0,0,0,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
18157,0,"Burrow, Joe",68.0,12.88,42,34,80.95,0,138.2,3,3,11.45,481,22,3,6.67,20,2022.0,7.0,25.317,CIN,0,ATL,0,0,0,0,0,0,0,1,1,0,0,0,35,17,23,10,39.2,0.0,0,0,0,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
18158,1,"Burrow, Joe",53.0,6.49,35,25,71.43,1,96.4,5,2,6.63,232,39,1,2.00,2,2022.0,8.0,25.325,CIN,@,CLE,0,0,0,0,0,0,0,1,0,1,0,0,13,32,31,10,14.5,0.0,0,0,0,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
18159,0,"Burrow, Joe",58.0,8.07,28,22,78.57,0,109.2,1,1,7.36,206,5,4,2.25,9,2022.0,9.0,25.331,CIN,0,CAR,0,0,0,0,0,0,0,1,1,0,0,0,42,21,6,11,19.1,0.0,0,0,0,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
18160,0,"Burrow, Joe",66.0,8.85,39,24,61.54,2,104.1,2,4,9.10,355,9,4,1.25,5,2022.0,11.0,25.345,CIN,@,PIT,0,0,0,0,0,0,0,1,1,0,0,0,37,30,20,11,28.7,0.0,0,0,0,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
